In [90]:
import os
import sys
import pandas as pd
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

%matplotlib inline
pylab.rcParams['figure.figsize'] = (12.0, 8.0)

In [91]:
CONN = os.environ.get('REDSHIFT_CONNECTION')
def query(sql):
    rs_con = psycopg2.connect(CONN)
    df = pd.read_sql(sql, con=rs_con)

    rs_con.close()

    return df

In [106]:
sql = """
with products as
(
    select
        p.product_id,
        p.product_set_name,
        p.product_set_category_group_name
    from
        da_dim.d_product_set p
    where p.product_set_category_group_name not in 
        ('Cufflinks & Studs', 'Accessory') and 
        p.product_set_name not in ('Cotton Dress Shirt')
)
select
    r.order_id,
    r.event_id,
    r.user_id,
    p.product_id,
    p.product_set_name as product_name,
    sum(r.checkout_item_cnt) as checkouts
from
    da_dim.f_checkouts_items r
    join
    products p on r.product_id = p.product_id
where
    r.item_is_valid = 1
    and
    r.event_date >=
        cast(date_trunc('week',
            convert_timezone('pst', getdate()) - Interval '53 Week') as date)
group by 1,2,3,4,5
order by r.order_id, p.product_id
"""

In [107]:
df = query(sql)

In [108]:
df.head()

,order_id,event_id,user_id,product_id,product_name,checkouts
0,4063,4101,81,57,Notch Lapel Tuxedo,1
1,4063,4101,81,229,Black Butterfly Bow Tie,1
2,12536,12359,270,59,Peak Lapel Tuxedo,1
3,12536,12359,270,73,Black Leather Wood Sole,1
4,12536,12359,270,229,Black Butterfly Bow Tie,1


In [109]:
basket = (df
          .groupby(['order_id', 'product_name'])['checkouts']
          .sum().unstack().reset_index().fillna(0)
          .set_index('order_id'))

In [110]:
basket.head()

product_name,Apricot Butterfly Bow Tie,Barre Pinstripe Tuxedo,Black & Tartan Tuxedo,Black & White Silk Repp Bow Tie,Black Belt,Black Butterfly Bow Tie,Black Cotton Necktie,Black Grosgrain Loafers,Black Holly Diamond Bow Tie,Black Knit Bow Tie,...,Velvet Jacket Tuxedo,Velvet Slip-On Shoes,Velvet Tuxedo Jacket,Violet Butterfly Bow Tie,White Dinner Jacket Tuxedo,White Linen Necktie,White Linen Wide Bow Tie,White Tuxedo Jacket,Wine Butterfly Bow Tie,Wing Collar Fly-Front Dress Shirt
order_id,,,,,,,,,,,,,,,,,,,,,
4063,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12536,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95953,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
434380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
531208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
basket.columns

Index(['Apricot Butterfly Bow Tie', 'Barre Pinstripe Tuxedo',
       'Black & Tartan Tuxedo', 'Black & White Silk Repp Bow Tie',
       'Black Belt', 'Black Butterfly Bow Tie', 'Black Cotton Necktie',
       'Black Grosgrain Loafers', 'Black Holly Diamond Bow Tie',
       'Black Knit Bow Tie',
       ...
       'Velvet Jacket Tuxedo', 'Velvet Slip-On Shoes', 'Velvet Tuxedo Jacket ',
       'Violet Butterfly Bow Tie', 'White Dinner Jacket Tuxedo',
       'White Linen Necktie', 'White Linen Wide Bow Tie',
       'White Tuxedo Jacket ', 'Wine Butterfly Bow Tie',
       'Wing Collar Fly-Front Dress Shirt'],
      dtype='object', name='product_name', length=180)

In [112]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [113]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [114]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [115]:
rules.head(20)

,antecedants,consequents,support,confidence,lift
0,(Black Leather Shoes),(Black Butterfly Bow Tie),0.120786,0.352642,1.479104
1,(Black Butterfly Bow Tie),(Black Leather Shoes),0.238416,0.178655,1.479104
2,(Black Low Cut Tuxedo Vest),(Black Butterfly Bow Tie),0.027916,0.604198,2.534219
3,(Black Butterfly Bow Tie),(Black Low Cut Tuxedo Vest),0.238416,0.070745,2.534219
4,(Black Patent Leather Shoes),(Black Butterfly Bow Tie),0.218492,0.482632,2.024326
5,(Black Butterfly Bow Tie),(Black Patent Leather Shoes),0.238416,0.442299,2.024326
6,(Black Tuxedo Vest),(Black Butterfly Bow Tie),0.025579,0.406017,1.702978
7,(Black Butterfly Bow Tie),(Black Tuxedo Vest),0.238416,0.043561,1.702978
8,(Cap Toe Shoes in Patent/Calf),(Black Butterfly Bow Tie),0.061671,0.465026,1.950481
9,(Black Butterfly Bow Tie),(Cap Toe Shoes in Patent/Calf),0.238416,0.120288,1.950481


In [89]:
rules[ (rules['lift'] >= 5) &
       (rules['confidence'] >= 0.2)].sort_values(by="lift", ascending=False)

,antecedants,consequents,support,confidence,lift
84,(Charcoal Suit Vest),(Charcoal Suit),0.022762,0.939516,10.334094
85,(Charcoal Suit),(Charcoal Suit Vest),0.090914,0.235226,10.334094
102,(Grey Cotton Necktie),(Grey Suit Vest),0.019239,0.532110,9.679879
194,"(Brown Leather Shoes, Grey Suit)",(Grey Suit Vest),0.046266,0.454296,8.264320
199,(Grey Suit Vest),"(Brown Leather Shoes, Grey Suit)",0.054971,0.382353,8.264320
33,(Black Low Cut Tuxedo Vest),(Cotton Wing Tip Shirt),0.027916,0.433991,7.699153
32,(Cotton Wing Tip Shirt),(Black Low Cut Tuxedo Vest),0.056369,0.214930,7.699153
192,(Brown Belt),"(Navy Suit, Brown Leather Shoes)",0.030846,0.363241,7.593962
189,"(Navy Suit, Brown Leather Shoes)",(Brown Belt),0.047833,0.234244,7.593962
109,(Navy Suit Vest),(Navy Suit),0.019197,0.929018,7.471761
